<a href="https://colab.research.google.com/github/ramahasiba/NLP/blob/langGraph/wishlist_management_agent_addition_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install langchain-community -q
!pip install langgraph -q
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.8 MB/s eta 0:

In [1]:
# HYBRID LANGGRAPH AGENT: RAG + ACTIONS (Wishlist)
# Assumptions:
# - You have a product CSV (e.g., products.csv)
# - Each row has: id, name, description, keywords, etc.
# - You're using Together.ai (LLaMA) and Chroma for vector DB

import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import Together
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal, Optional

ModuleNotFoundError: Module langchain_community.embeddings not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [8]:
# --- CONFIGURATION ---
TOGETHER_API_KEY ="2755374615477ce7aeaf2ac17491100ff51b9aa570682316d7656b8b91fb17c4"

# --- 1. LOAD PRODUCT DATA ---
loader = CSVLoader(file_path="products.csv")
documents = loader.load()

# --- 2. SPLIT + EMBED + STORE ---
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

<ipython-input-8-462000752>:11: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [9]:
vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()

In [11]:
# --- 3. LLaMA LLM via Together ---
llm = Together(
    model="meta-llama/Llama-3-8b-chat-hf",
    temperature=0.56,
    together_api_key=TOGETHER_API_KEY,
)

In [12]:
# --- 4. RAG Chain ---
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [13]:
# --- 5. Wishlist Action Tool ---
def add_to_wishlist(product_id: str) -> str:
    import requests
    url = "https://radixtechs.com/ai_store/index.php?route=account/wishlist/add"
    data = {"product_id": product_id}
    headers = {
        "Cookie": "OCSESSID=47c16eb9ed5d76e9d92098d902; language=en-gb; currency=USD"
    }
    try:
        response = requests.post(url, data=data, headers=headers)
        if response.status_code == 200:
            return f"✅ Product {product_id} added to wishlist."
        else:
            return f"❌ Failed to add product {product_id}."
    except Exception as e:
        return f"Error: {str(e)}"


In [14]:
wishlist_tool = Tool(
    name="add_to_wishlist",
    func=lambda x: add_to_wishlist(x.get("product_id", "")),
    description="Adds a product to the user's wishlist. Requires product_id."
)

In [15]:
# --- 6. STATE MANAGEMENT ---
class AgentState(TypedDict):
    input: str
    route: Literal["rag", "wishlist"]
    result: Optional[str]

In [16]:
# --- 7. ROUTER FUNCTION ---
def router(state: AgentState) -> str:
    user_input = state["input"].lower()
    if "add to wishlist" in user_input or "add" in user_input:
        return "wishlist"
    else:
        return "rag"

In [17]:
# --- 8. NODES ---
def rag_node(state: AgentState) -> AgentState:
    answer = qa_chain.run(state["input"])
    return {**state, "result": answer}

def wishlist_node(state: AgentState) -> AgentState:
    # Extract ID (this is just a mock rule, refine as needed)
    import re
    product_id = re.findall(r'\d+', state["input"])
    pid = product_id[0] if product_id else ""
    result = wishlist_tool.func({"product_id": pid})
    return {**state, "result": result}

In [ ]:
# --- 9. BUILD LANGGRAPH ---
graph = StateGraph(AgentState)
graph.add_node("rag", rag_node)
graph.add_node("wishlist", wishlist_node)
graph.set_entry_point("router")
graph.add_conditional_edges("router", router, {"rag": "rag", "wishlist": "wishlist"})
graph.add_edge("rag", END)
graph.add_edge("wishlist", END)
graph.add_node("router", lambda x: x)
app = graph.compile()

In [20]:
# --- 10. RUN EXAMPLES ---
examples = [
    {"input": "What is the material of product with id 23?"},
    {"input": "Add product 30 to my wishlist"}
]

for ex in examples:
    output = app.invoke(ex)
    print("\nUser Input:", ex["input"])
    print("Agent Response:", output["result"])



User Input: What is the material of product with id 23?
Agent Response:  I don't know. There is no product with id 23. The provided context only includes information about products with ids 10, 12, 27, and 30. There is no information about a product with id 23. Therefore, I cannot provide an answer to this question.

User Input: Add product 30 to my wishlist
Agent Response: ✅ Product 30 added to wishlist.
